# Custom Components

This is a guide on how to build a simple app and custom component spec
and launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2022-03-04 08:51:16 INFO     Log files located in: /tmp/torchx_truany00/torchx/hello_world-tk10cntrqcq9nd/greeter/0


torchx 2022-03-04 08:51:17 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2022-03-04 08:51:18 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-tk10cntrqcq9nd


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [4]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

Step 1/2 : FROM ghcr.io/pytorch/torchx:0.1.0rc1


0.1.0rc1: Pulling from pytorch/torchx


4bbfd2c87b75: Pulling fs layer


d2e110be24e1: Pulling fs layer


889a7173dcfe: Pulling fs layer


...

 ---> 3dbec59e8049


Step 2/2 : ADD my_app.py .


 ---> 8db642e87daa


Successfully built 8db642e87daa


Successfully tagged my_app:latest


We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2022-03-04 08:53:35 INFO     Building workspace: file:///home/runner/work/torchx/torchx/docs/source for role[0]: greeter, image: my_app:latest


torchx 2022-03-04 08:53:36 WARNING  failed to pull image my_app:latest, falling back to local: 404 Client Error for http+docker://localhost/v1.41/images/create?tag=latest&fromImage=my_app: Not Found ("pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")


torchx 2022-03-04 08:53:44 INFO     Done building workspace


torchx 2022-03-04 08:53:44 INFO     New image: sha256:9b7bb4429767e3f7682d180fc1955c3196413a8af7650d933d46db6d083fcc80 built from workspace


torchx 2022-03-04 08:53:46 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2022-03-04 08:53:47 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-nfh2nmtlf9hvv


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [7]:
%%sh
torchx builtins

Found 10 builtin components:


  1. dist.ddp


  2. serve.torchserve


  3. utils.binary


  4. utils.booth


...

  6. utils.echo


  7. utils.python


  8. utils.sh


  9. utils.touch


 10. metrics.tensorboard


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [8]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2022-03-04 08:53:50 INFO     Building workspace: file:///home/runner/work/torchx/torchx/docs/source for role[0]: echo, image: ghcr.io/pytorch/torchx:0.1.2dev0


torchx 2022-03-04 08:56:05 INFO     Done building workspace


torchx 2022-03-04 08:56:05 INFO     New image: sha256:21edff9d24b6f101c9ca79eb6e1dabecfb2707a02752eb40e878b42cb1f30f8e built from workspace


torchx 2022-03-04 08:56:06 INFO     Waiting for the app to finish...


torchx 2022-03-04 08:56:07 INFO     Job finished: SUCCEEDED


echo/0 Hello :)


local_docker://torchx/echo-tq2kfssr5zw9kc
